<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Les-imports-et-configuration-du-document" data-toc-modified-id="Les-imports-et-configuration-du-document-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Les imports et configuration du document</a></span><ul class="toc-item"><li><span><a href="#La-lecture-des-fichiers-d'un-répértoire" data-toc-modified-id="La-lecture-des-fichiers-d'un-répértoire-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>La lecture des fichiers d'un répértoire</a></span></li><li><span><a href="#La-description-du-dataframe" data-toc-modified-id="La-description-du-dataframe-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>La description du dataframe</a></span></li></ul></li></ul></div>

# Les imports et configuration du document

In [1]:
import pandas as pd 
import seaborn as sns
import warnings 
import os
from datetime import datetime
warnings.filterwarnings(action="ignore")

%pylab inline

plt.style.use('seaborn-darkgrid')
sns.set(font_scale=3)

os.chdir(r"C:\SimplonIA\donnees")

Populating the interactive namespace from numpy and matplotlib


## La lecture des fichiers d'un répértoire 

In [2]:
for dirname, _, filenames in os.walk('meteo'):
    for filename in filenames:
        if filename.rfind('synop') >= 0 :
            print(os.path.join(dirname, filename))

meteo\synop.199601.csv
meteo\synop.199602.csv
meteo\synop.199603.csv
meteo\synop.199604.csv
meteo\synop.199605.csv
meteo\synop.199606.csv
meteo\synop.199607.csv
meteo\synop.199608.csv
meteo\synop.199609.csv
meteo\synop.199610.csv
meteo\synop.199611.csv
meteo\synop.199612.csv
meteo\synop.199701.csv
meteo\synop.199702.csv
meteo\synop.199703.csv
meteo\synop.199704.csv
meteo\synop.199705.csv
meteo\synop.199706.csv
meteo\synop.199707.csv
meteo\synop.199708.csv
meteo\synop.199709.csv
meteo\synop.199710.csv
meteo\synop.199711.csv
meteo\synop.199712.csv
meteo\synop.199801.csv
meteo\synop.199802.csv
meteo\synop.199803.csv
meteo\synop.199804.csv
meteo\synop.199805.csv
meteo\synop.199806.csv
meteo\synop.199807.csv
meteo\synop.199808.csv
meteo\synop.199809.csv
meteo\synop.199810.csv
meteo\synop.199811.csv
meteo\synop.199812.csv
meteo\synop.199901.csv
meteo\synop.199902.csv
meteo\synop.199903.csv
meteo\synop.199904.csv
meteo\synop.199905.csv
meteo\synop.199906.csv
meteo\synop.199907.csv
meteo\synop

In [3]:
def dateparse(d):
    return pd.datetime.strptime(d,'%Y%m%d%H%M%S')

meteo = pd.DataFrame()

for dirname, _, filenames in os.walk('meteo'):
    for filename in filenames:
        if filename.rfind('synop') >= 0 :
            meteo = pd.concat([meteo,
                               pd.read_csv(os.path.join(dirname, filename),
                                           sep=';',
                                           usecols=['numer_sta','date','dd','ff','t','u','vv',
                                                    'pres','rr1','rr3','rr6','rr12','rr24'],
                                           na_values='mq',
                                           dtype={'numer_sta':str,'date':str},
                                           parse_dates={'DateHeure': ['date']},
                                           date_parser=dateparse
                                          ).rename( columns={'numer_sta':'Station',
                                                              'date':'Date',
                                                              'dd':'DirectionVent',
                                                              'ff':'VitesseVent',
                                                              't':'Temperature',
                                                              'u':'Humidite',
                                                              'vv':'Visibilite',
                                                              'pres':'Pression',
                                                              'rr1' :'Precipitation01',
                                                              'rr3' :'Precipitation03',
                                                              'rr6' :'Precipitation06',
                                                              'rr12':'Precipitation12',
                                                              'rr24':'Precipitation24'})],axis=0)
            
meteo.Temperature = meteo.Temperature - 273.15
meteo.Pression    = meteo.Pression / 100
meteo.Visibilite  = meteo.Visibilite / 1000
meteo.head(5)            

,Station,Date,DirectionVent,VitesseVent,Temperature,Humidite,Visibilite,Pression,Precipitation01,Precipitation03,Precipitation06,Precipitation12,Precipitation24
0,07005,19960101000000,170.0,2.0,2.89,99.0,NaN,990.9,NaN,NaN,0.0,NaN,NaN
1,07015,19960101000000,0.0,0.0,2.80,96.0,0.3,993.8,NaN,NaN,0.0,NaN,NaN
2,07020,19960101000000,180.0,8.2,8.50,99.0,NaN,995.7,NaN,NaN,0.0,NaN,NaN
3,07027,19960101000000,120.0,4.1,5.89,100.0,1.2,991.3,NaN,0.0,0.0,NaN,NaN
4,07037,19960101000000,190.0,1.0,5.39,98.0,NaN,981.7,NaN,0.2,0.2,NaN,NaN


In [4]:
postes = pd.read_csv('meteo/postesSynop.csv',sep=';',dtype={'ID':str})
postes.Nom =postes.Nom.apply(lambda x : x if x in ['CLERMONT-FD','MONT-DE-MARSAN',
                                       'ST-PIERRE','ST-BARTHELEMY METEO'] 
                               else x[0:x.find('-')] 
                                    if x.find('-') != -1 else x).apply(lambda x : str(x).title())
postes.Altitude = postes.Altitude.astype('int16')
postes = postes[postes.ID < '08000']
meteo = postes.merge(meteo, how = "inner", 
                     left_on = "ID", right_on = "Station").drop(["ID","Station"], axis = "columns")

In [7]:
meteo.head()

,Nom,Latitude,Longitude,Altitude,Date,DirectionVent,VitesseVent,Temperature,Humidite,Visibilite,Pression,Precipitation01,Precipitation03,Precipitation06,Precipitation12,Precipitation24
0,Abbeville,50.136,1.834,69,19960101000000,170.0,2.0,2.89,99.0,NaN,990.9,NaN,NaN,0.0,NaN,NaN
1,Abbeville,50.136,1.834,69,19960101030000,130.0,3.0,2.70,99.0,NaN,990.7,NaN,0.2,NaN,NaN,NaN
2,Abbeville,50.136,1.834,69,19960101060000,120.0,3.0,2.89,99.0,0.1,990.9,NaN,0.0,NaN,0.0,NaN
3,Abbeville,50.136,1.834,69,19960101090000,130.0,4.1,3.60,99.0,0.8,991.1,NaN,0.0,NaN,0.0,NaN
4,Abbeville,50.136,1.834,69,19960101120000,100.0,5.1,4.39,99.0,1.6,990.4,NaN,0.0,0.0,NaN,NaN


In [6]:
meteo.to_csv(r'C:\SimplonIA\donnees\\meteo.csv', index=None, index_label=None)

In [7]:
meteo.to_parquet(r'C:\SimplonIA\donnees\\meteo.gzip',compression='gzip', engine='pyarrow')

In [8]:
%%time
pd.read_csv(r'C:\SimplonIA\donnees\\meteo.csv').dtypes

Wall time: 3.9 s


Nom                 object
Latitude           float64
Longitude          float64
Altitude             int64
DateHeure           object
DirectionVent      float64
VitesseVent        float64
Temperature        float64
Humidite           float64
Visibilite         float64
Pression           float64
Precipitation01    float64
Precipitation03    float64
Precipitation06    float64
Precipitation12    float64
Precipitation24    float64
dtype: object

In [17]:
%%time
meteo = pd.read_parquet(r'C:\SimplonIA\donnees\\meteo.gzip', engine='pyarrow')

Wall time: 431 ms


## La description du dataframe

In [8]:
meteo.shape

(2960726, 16)

In [18]:
meteo.dtypes

Nom                        object
Latitude                  float64
Longitude                 float64
Altitude                    int16
DateHeure          datetime64[ns]
DirectionVent             float64
VitesseVent               float64
Temperature               float64
Humidite                  float64
Visibilite                float64
Pression                  float64
Precipitation01           float64
Precipitation03           float64
Precipitation06           float64
Precipitation12           float64
Precipitation24           float64
dtype: object

In [19]:
meteo[['Precipitation01','Precipitation03',
       'Precipitation06','Precipitation12',
       'Precipitation24']].bfill(axis=1).iloc[:, 0].isna().sum() 

19010

In [20]:
meteo['Precipitation'] =  meteo['Precipitation03'].combine_first(meteo['Precipitation06']/2)\
                                                  .combine_first(meteo['Precipitation12']/4)\
                                                  .combine_first(meteo['Precipitation24']/8)\
                                                  .combine_first(meteo['Precipitation01']*3)

In [21]:
meteo['aj'] = meteo.DateHeure.dt.year * 1000 + meteo.DateHeure.dt.dayofyear
meteo['mois'] = meteo.DateHeure.dt.month

In [22]:
meteo.head(10)

,Nom,Latitude,Longitude,Altitude,DateHeure,DirectionVent,VitesseVent,Temperature,Humidite,Visibilite,Pression,Precipitation01,Precipitation03,Precipitation06,Precipitation12,Precipitation24,Precipitation,aj,mois
0,Abbeville,50.136,1.834,69,1996-01-01 00:00:00,170.0,2.0,2.89,99.0,NaN,99.09,NaN,NaN,0.0,NaN,NaN,0.0,1996001,1
1,Abbeville,50.136,1.834,69,1996-01-01 03:00:00,130.0,3.0,2.70,99.0,NaN,99.07,NaN,0.2,NaN,NaN,NaN,0.2,1996001,1
2,Abbeville,50.136,1.834,69,1996-01-01 06:00:00,120.0,3.0,2.89,99.0,0.1,99.09,NaN,0.0,NaN,0.0,NaN,0.0,1996001,1
3,Abbeville,50.136,1.834,69,1996-01-01 09:00:00,130.0,4.1,3.60,99.0,0.8,99.11,NaN,0.0,NaN,0.0,NaN,0.0,1996001,1
4,Abbeville,50.136,1.834,69,1996-01-01 12:00:00,100.0,5.1,4.39,99.0,1.6,99.04,NaN,0.0,0.0,NaN,NaN,0.0,1996001,1
5,Abbeville,50.136,1.834,69,1996-01-01 15:00:00,130.0,3.0,4.30,99.0,0.1,99.07,NaN,NaN,0.0,NaN,NaN,0.0,1996001,1
6,Abbeville,50.136,1.834,69,1996-01-01 18:00:00,100.0,4.1,3.39,99.0,NaN,99.16,NaN,0.0,NaN,0.0,NaN,0.0,1996001,1
7,Abbeville,50.136,1.834,69,1996-01-01 21:00:00,90.0,4.1,3.60,99.0,NaN,99.17,NaN,NaN,NaN,0.0,NaN,0.0,1996001,1
8,Abbeville,50.136,1.834,69,1996-01-02 00:00:00,90.0,4.1,3.80,99.0,NaN,99.27,NaN,NaN,0.0,NaN,NaN,0.0,1996002,1
9,Abbeville,50.136,1.834,69,1996-01-02 03:00:00,80.0,5.1,3.50,99.0,NaN,99.41,NaN,NaN,0.0,NaN,NaN,0.0,1996002,1


In [23]:
meteo.pivot_table(index=['Nom', 'Latitude', 'Longitude', 'Altitude'],
                  columns='mois',
                  values='Temperature').head()

,,,mois,1,2,3,4,5,6,7,8,9,10,11,12
Nom,Latitude,Longitude,Altitude,,,,,,,,,,,,
Abbeville,50.136000,1.834000,69,4.372309,4.945650,7.092692,9.841288,12.885313,15.764326,17.569472,17.764522,15.187675,11.868993,7.689304,5.042880
Ajaccio,41.918000,8.792667,5,8.965797,8.983649,10.928321,13.645100,17.333600,21.348559,23.678523,23.791242,20.658899,17.437811,13.049093,9.909476
Alencon,48.445500,0.110167,143,4.460286,4.931409,7.306194,10.051220,13.388578,16.758878,18.527981,18.375294,15.269251,11.773452,7.560239,4.879053
Bale,47.614333,7.510000,263,2.006641,3.041258,6.548861,10.649915,14.845584,18.637804,20.016305,19.554766,15.222541,10.951525,5.886351,2.770062
Bastia,42.540667,9.485167,10,9.135010,9.186098,11.185869,13.839347,17.752433,21.957800,24.680482,24.734411,21.067004,17.364209,13.101640,10.072712


In [24]:
df = meteo.pivot_table(index=['Nom', 'Latitude', 'Longitude', 'Altitude'],
                  columns='mois',
                  values=['VitesseVent','Temperature','Humidite','Visibilite','Pression','Precipitation'])

In [25]:
df.head()

Humidite                        \
mois                                           1          2          3    
Nom       Latitude  Longitude Altitude                                    
Abbeville 50.136000 1.834000  69        86.643851  83.495916  80.872895   
Ajaccio   41.918000 8.792667  5         77.536653  75.814894  74.708023   
Alencon   48.445500 0.110167  143       88.779036  84.219139  79.774801   
Bale      47.614333 7.510000  263       84.562016  79.261563  73.540492   
Bastia    42.540667 9.485167  10        73.501136  71.194632  71.902730   

                                                                         \
mois                                           4          5          6    
Nom       Latitude  Longitude Altitude                                    
Abbeville 50.136000 1.834000  69        75.789491  78.272565  78.693239   
Ajaccio   41.918000 8.792667  5         74.531972  73.383379  70.672541   
Alencon   48.445500 0.110167  143       74.728751  75.960862  73.868152   
Bale      47.614333 7.510000  263       69.511861  72.286410  71.299565   
Bastia    42.540667 9.485167  10        73.772197  72.486013  70.153537   

                                                                         \
mois                                           7          8          9    
Nom       Latitude  Longitude Altitude                                    
Abbeville 50.136000 1.834000  69        77.665528  78.123984  80.398890   
Ajaccio   41.918000 8.792667  5         67.782433  67.758247  70.476166   
Alencon   48.445500 0.110167  143       71.413787  72.949763  76.686011   
Bale      47.614333 7.510000  263       70.733161  74.412155  79.186293   
Bastia    42.540667 9.485167  10        66.395288  66.577640  68.896697   

                                                   ... VitesseVent            \
mois                                           10  ...          3         4    
Nom       Latitude  Longitude Altitude             ...                         
Abbeville 50.136000 1.834000  69        84.526476  ...    4.837462  4.310927   
Ajaccio   41.918000 8.792667  5         74.684922  ...    3.263772  3.298995   
Alencon   48.445500 0.110167  143       84.737040  ...    3.682880  3.416779   
Bale      47.614333 7.510000  263       84.042173  ...    2.881263  2.682888   
Bastia    42.540667 9.485167  10        76.586406  ...    3.157783  2.891219   

                                                                      \
mois                                          5         6         7    
Nom       Latitude  Longitude Altitude                                 
Abbeville 50.136000 1.834000  69        4.241158  3.913799  3.965724   
Ajaccio   41.918000 8.792667  5         3.322228  3.458372  3.617263   
Alencon   48.445500 0.110167  143       3.427330  3.235215  3.272902   
Bale      47.614333 7.510000  263       2.661831  2.450710  2.396936   
Bastia    42.540667 9.485167  10        2.644041  2.611262  2.753413   

                                                                      \
mois                                          8         9         10   
Nom       Latitude  Longitude Altitude                                 
Abbeville 50.136000 1.834000  69        3.751216  3.794631  4.314726   
Ajaccio   41.918000 8.792667  5         3.519628  3.399916  3.098358   
Alencon   48.445500 0.110167  143       2.969542  2.921988  3.080628   
Bale      47.614333 7.510000  263       2.133560  2.162878  2.298499   
Bastia    42.540667 9.485167  10        2.705798  2.819304  2.771190   

                                                            
mois                                          11        12  
Nom       Latitude  Longitude Altitude                      
Abbeville 50.136000 1.834000  69        4.462587  4.809195  
Ajaccio   41.918000 8.792667  5         3.214738  3.249873  
Alencon   48.445500 0.110167  143       3.361686  3.601605  
Bale      47.614333 7.510000  263       2.496798  2.862024  
Bastia    42.540